## Setting up SVM

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

In [27]:
dataPath = '../data/acousticbrainz-mediaeval-discogs-train.tsv'
data = pd.read_csv(dataPath, delimiter='\t')

/var/folders/55/m9y_rpf97z3cqtjh65x7xy8c0000gn/T/ipykernel_74413/3846289340.py:2: DtypeWarning: Columns (17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dataPath, delimiter='\t')


# the column names in the data file

In [23]:
print(data.keys())

Index(['recordingmbid', 'releasegroupmbid', 'genre1', 'genre2', 'genre3',
       'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10',
       'genre11', 'genre12', 'genre13', 'genre14', 'genre15', 'genre16',
       'genre17', 'genre18', 'genre19', 'genre20', 'genre21', 'genre22',
       'genre23', 'genre24', 'genre25', 'genre26', 'genre27', 'genre28',
       'genre29', 'genre30'],
      dtype='object')


In [ ]:
# depending on how the data is set up we will need to figure out how to classify the genre
# keeping it general for now
data['is_rnb'] = data['genre'].apply(lambda x: 1 if x == 'R&B' else 0)

# once we decide which columns we want to drop from our features
dropCols = ['genre', 'is_rnb']
X = data.drop(columns=dropCols)

# what we want to predict
y = data['is_rnb']

# train-test split of 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (important for SVM)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# there are many kernal options, we can choose from 
# {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable
svm_model = SVC(kernel='linear') 

# fit the model and predict
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

# output results
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# functions to plot the trained model with X and y

In [28]:
# this graphing of the model comes from Alex Thomo's SENG 474 class

def plot_points(features, labels):
    X = np.array(features)
    y = np.array(labels)
    spam = X[np.argwhere(y==1)]
    ham = X[np.argwhere(y==0)]
    plt.scatter([s[0][0] for s in spam],
                   [s[0][1] for s in spam],
                   s = 35,
                   color = 'cyan',
                   edgecolor = 'k',
                   marker = '^')
    plt.scatter([s[0][0] for s in ham],
                   [s[0][1] for s in ham],
                   s = 25,
                   color = 'red',
                   edgecolor = 'k',
                   marker = 's')
    plt.xlabel('x_1')
    plt.ylabel('x_2')
    plt.legend(['label 1','label 0'])

def plot_model(X, y, model):
    X = np.array(X)
    y = np.array(y)
    plot_step = 0.01
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,colors = 'k',linewidths = 3)
    plot_points(X, y)
    plt.contourf(xx, yy, Z, colors=['red', 'blue'], alpha=0.2, levels=range(-1,2))
    plt.show()